# DATA

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

np.set_printoptions(precision=6, suppress=True)

df_train = pd.read_csv('/content/drive/Shareddrives/Dacon/train/train.csv')
submission = pd.read_csv('/content/drive/Shareddrives/Dacon/sample_submission.csv')

dfs_test = []

for i in range(81):
    temp_df = pd.read_csv(f"/content/drive/Shareddrives/Dacon/test/{i}.csv")
    dfs_test.append(temp_df.copy())

In [ ]:
import math
    
def Make_New_Features(df_train):
    df_train['GHI'] = df_train['DHI'] + df_train['DNI']
    df_train['RDHNI'] = df_train['GHI'].apply(lambda x : x**0.5)
#====================================================================================
    df_train['Lower_Cloud'] = 2.5 * df_train['RH'] / 100 - 1.5
    df_train['Middle_Cloud'] = 4 * df_train['RH'] / 100 - 3.0
#====================================================================================
    b = 17.62
    c = 243.12
    df_train['gamma'] = (b * df_train['T'] /(c + df_train['T'])) + (df_train['RH'] / 100.0).apply(lambda x: math.log(x + 1))
    df_train['DD'] = (c * df_train['gamma']) / (b - df_train['gamma'])

    df_train.drop('gamma', axis=1, inplace=True)
#====================================================================================
        
    return df_train

In [ ]:
from sklearn import preprocessing

scalers ={}

def preprocess(df, make_scaler=True):
    df = df.copy()
    df = Make_New_Features(df)
    
    df['Minute'].replace(30, 0.5, inplace=True)
    df['Time'] = df['Hour'] + df['Minute']
    
    df['Time_x'] = np.cos((df['Time'] * 2 * np.pi) / 24.0)
    df['Time_y'] = np.sin((df['Time'] * 2 * np.pi) / 24.0)
    
    scale_target_columns = ['TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                           'Middle_Cloud', 'DD']
    
    temp_df_agg = pd.DataFrame()
    for c in scale_target_columns:
        temp_df_agg[f'max_week_{c}'] = df.groupby('Day')[c].max().rolling(7, center=True).max()
        temp_df_agg[f'min_week_{c}'] = df.groupby('Day')[c].min().rolling(7, center=True).min()

        temp_df_agg[f'max_week_{c}'].iloc[:3] = temp_df_agg.iloc[3][f'max_week_{c}']
        temp_df_agg[f'max_week_{c}'].iloc[-3:] = temp_df_agg.iloc[-4][f'max_week_{c}']
        temp_df_agg[f'min_week_{c}'].iloc[:3] = temp_df_agg.iloc[3][f'min_week_{c}']
        temp_df_agg[f'min_week_{c}'].iloc[-3:] = temp_df_agg.iloc[-4][f'min_week_{c}']
        
        
    temp_df_agg = temp_df_agg.reset_index()
    df = pd.merge(df, temp_df_agg, left_on='Day', right_on='Day')
    for c in scale_target_columns:
        df[f'scaled_{c}'] = (df[c] - df[f'min_week_{c}'] + 1) / (df[f'max_week_{c}']+1)

    df['peak_diff'] = df['max_week_TARGET'] - df['TARGET']
    df['peak_diff_scaled'] = df['peak_diff'] / df['max_week_TARGET']
    
    normalize_target_column = ['DHI', 'DNI', 'WS', 'RH', 'T', 'GHI', 'RDHNI', 'TARGET',
                               'Lower_Cloud', 'Middle_Cloud', 'DD']
    
    for c in normalize_target_column:
        x = df[[c]].copy() #returns a numpy array
        if make_scaler:
            std_scaler = preprocessing.MinMaxScaler()
            std_scaler.fit(x)
            df[c] = std_scaler.transform(x)
            scalers[c] = std_scaler
        else:
            df[c] = scalers[c].transform(x)
    
    df = df.drop(['Hour', 'Minute', 'Time'], axis=1)
    
    return df

df = preprocess(df_train)

In [ ]:
temp = []
for df_test in dfs_test:
    temp.append(preprocess(df_test, make_scaler=False))
dfs = temp

In [ ]:
def make_train_data_set(df):
    df = df.astype('float')
    ins_datas = df[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                   'Middle_Cloud', 'DD', 'Time_x', 'Time_y',
                   'scaled_TARGET', 'peak_diff_scaled']].values
    outs_datas = df[['TARGET']].values
    
    ins = []
    outs = []
    
    for i in range(len(df)-48*9):
        ins.append(ins_datas[i:i+48*7])
        outs.append(outs_datas[i+48*8:i+48*9])
        
    return np.array(ins), np.array(outs)

def make_prediction_data_set(df):
    df = df.astype('float')
    ins_datas = df[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'GHI', 'RDHNI', 'Lower_Cloud',
                   'Middle_Cloud', 'DD', 'Time_x', 'Time_y',
                   'scaled_TARGET', 'peak_diff_scaled']].values
    
    ins = [ins_datas]

    return np.array(ins)

ins, outs = make_train_data_set(df)
ins = ins.reshape(ins.shape + (1,))
ins.shape, outs.shape

((52128, 336, 15, 1), (52128, 48, 1))

In [ ]:
term = 17376

x_year_1 = ins[:term]
x_year_2 = ins[term:term*2]
x_year_3 = ins[term*2:]

y_year_1 = outs[:term]
y_year_2 = outs[term:term*2]
y_year_3 = outs[term*2:]

drop_columns=[]
x_year_1.shape, x_year_2.shape, x_year_3.shape, y_year_1.shape, y_year_2.shape, y_year_3.shape

((17376, 336, 15, 1),
 (17376, 336, 15, 1),
 (17376, 336, 15, 1),
 (17376, 48, 1),
 (17376, 48, 1),
 (17376, 48, 1))

In [ ]:
temp = []
for df in dfs:
    temp.append(make_prediction_data_set(df))
pre_ins = np.concatenate(temp)
pre_ins = pre_ins.reshape(pre_ins.shape + (1, ))
pre_ins.shape

(81, 336, 15, 1)

# Model


In [ ]:
# stacked generalization with neural net meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model 
from keras.layers import Input
from keras.layers import Dense
from numpy import argmax
import keras

In [ ]:
# Custom Function
def pinball_loss(tau):
    def loss(y_true, y_pred):
        err = y_true - y_pred
        return K.mean(K.maximum(tau * err, (tau - 1) * err))
    return loss

def load_cnn(j, lstm=False):
  Qauntiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

  Model_year = []

  for k in range(3):      # [Year1, Year2]-Model1, [Year1, Year3]-Model2, [Year2, Year3]-Model3
    Model_Quan = []
    for i in Qauntiles:    # Qauntiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
      if lstm :
        filepath = '/content/drive/Shareddrives/Dacon/models/ensemble/model_' + str(k+1) + '_lstm_1_8_' + str(i) + '_' + str(j)
      else :
        filepath = '/content/drive/Shareddrives/Dacon/models/ensemble/model_' + str(k+1) + '_cnn_1_8_' + str(i) + '_' + str(j)
      model = keras.models.load_model(filepath, custom_objects={'loss': pinball_loss(i)})
      Model_Quan.append(model)
      print(filepath)

    Model_year.append(Model_Quan)
  
    # else:
    #   CNN_DAY2.append(Model1)
    #   CNN_DAY2.append(Model2)
    #   CNN_DAY2.append(Model3)
  #CNN.append(CNN_DAY2)

  return Model_year

# j : day
# k : model1,2,3
def load_lstm(j, k):
  Qauntiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

  #Model_year = []

  Model_Quan = []
  for i in Qauntiles:    # Qauntiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    filepath = '/content/drive/Shareddrives/Dacon/models/ensemble/model_' + str(k) + '_lstm_1_8_' + str(i) + '_' + str(j)
    
    model = keras.models.load_model(filepath, custom_objects={'loss': pinball_loss(i)})
    Model_Quan.append(model)
    print(filepath)
  
    # else:
    #   CNN_DAY2.append(Model1)
    #   CNN_DAY2.append(Model2)
    #   CNN_DAY2.append(Model3)
  #CNN.append(CNN_DAY2)

  return Model_Quan

def load_lstm_fine(j, k):
  Qauntiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

  #Model_year = []

  Model_Quan = []
  for i in Qauntiles:    # Qauntiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    filepath = '/content/drive/Shareddrives/Dacon/models/ensemble/model_' + str(k) + '_lstm_1_8_' + str(i) + '_' + str(j) + '_fined'
    
    model = keras.models.load_model(filepath, custom_objects={'loss': pinball_loss(i)})
    Model_Quan.append(model)
    print(filepath)
  
    # else:
    #   CNN_DAY2.append(Model1)
    #   CNN_DAY2.append(Model2)
    #   CNN_DAY2.append(Model3)
  #CNN.append(CNN_DAY2)

  return Model_Quan
#CNN_Day1 = load_cnn(1, lstm=False)
#CNN_Day2 = load_cnn(2, lstm=False)
#LSTM_Day1 = load_cnn(1, lstm=True)
#LSTM_Day2 = load_cnn(2, lstm=True)

In [ ]:
CNN_Model1 = CNN_Day2[0]
CNN_Model2 = CNN_Day2[1]
CNN_Model3 = CNN_Day2[2]

cnn_quantile_predict_x = []
cnn_qauntile_predict_y = []

for i in range(9):
  year1 = CNN_Model3[i].predict(x_year_1)
  year2 = CNN_Model2[i].predict(x_year_2)
  year3 = CNN_Model1[i].predict(x_year_3)

  y_year_1 = y_year_1.reshape(17376, 48)
  y_year_2 = y_year_2.reshape(17376, 48)
  y_year_3 = y_year_3.reshape(17376, 48)

  x_year = np.vstack([year1, year2, year3])
  y_year = np.vstack((y_year_1, y_year_2, y_year_3))

  cnn_quantile_predict_x.append(x_year)
  cnn_qauntile_predict_y.append(y_year)

In [ ]:
import pickle
with open("/content/drive/Shareddrives/Dacon/data/cnn_2.pickle", "wb") as fw:
  pickle.dump(cnn_quantile_predict_x, fw)
  pickle.dump(cnn_qauntile_predict_y, fw)

In [ ]:
# j : day
# k : model1,2,3

#lstm_model = load_lstm(1, 1)
#lstm_model = load_lstm(1, 2)
#lstm_model = load_lstm(1, 3)
#lstm_model = load_lstm(2, 1)
#lstm_model = load_lstm(2, 2)
#lstm_model = load_lstm(2, 3)

lstm_model = load_lstm_fine(1, 1)
#lstm_model = load_lstm(1, 2)
#lstm_model = load_lstm(1, 3)
#lstm_model = load_lstm(2, 1)
#lstm_model = load_lstm(2, 2)
#lstm_model = load_lstm(2, 3)

In [ ]:
CNN_Model1 = lstm_model
#CNN_Model2 = lstm_model
#CNN_Model3 = lstm_model

cnn_quantile_predict_x = []
cnn_qauntile_predict_y = []

for i in range(9):
  # year1 = CNN_Model3[i].predict(x_year_1)
  #year2 = CNN_Model2[i].predict(x_year_2)
  year3 = CNN_Model1[i].predict(x_year_3)

  # y_year_1 = y_year_1.reshape(17376, 48)
  #y_year_2 = y_year_2.reshape(17376, 48)
  y_year_3 = y_year_3.reshape(17376, 48)

  #x_year = np.vstack([year1, year2, year3])
  #y_year = np.vstack((y_year_1, y_year_2, y_year_3))

  cnn_quantile_predict_x.append(year3)
  cnn_qauntile_predict_y.append(y_year_3)

In [ ]:
cnn_quantile_predict_x[0].shape

(17376, 48)

In [ ]:
import pickle
with open("/content/drive/Shareddrives/Dacon/data/lstm_fine_1_3.pickle", "wb") as fw:
  pickle.dump(cnn_quantile_predict_x, fw)
  pickle.dump(cnn_qauntile_predict_y, fw)

In [ ]:
with open('/content/drive/Shareddrives/Dacon/data/lstm_1_1.pickle', 'rb') as fr:
  X_1_1 = pickle.load(fr)
  Y_1_1 = pickle.load(fr)
with open('/content/drive/Shareddrives/Dacon/data/lstm_1_2.pickle', 'rb') as fr:
  X_1_2 = pickle.load(fr)
  Y_1_2 = pickle.load(fr)
with open('/content/drive/Shareddrives/Dacon/data/lstm_1_3.pickle', 'rb') as fr:
  X_1_3 = pickle.load(fr)
  Y_1_3 = pickle.load(fr)

In [ ]:
np.hstack([X_1_1, X_1_2, X_1_3]).shape

(9, 52128, 48)

In [ ]:
x_year = np.hstack([X_1_1, X_1_2, X_1_3])
y_year = np.hstack([Y_1_1, Y_1_2, Y_1_3])

In [ ]:
x_year.shape, y_year.shape

((9, 52128, 48), (9, 52128, 48))

In [ ]:
import pickle
with open("/content/drive/Shareddrives/Dacon/data/lstm_1.pickle", "wb") as fw:
  pickle.dump(x_year, fw)
  pickle.dump(y_year, fw)

# Train DATA

In [ ]:
with open('/content/drive/Shareddrives/Dacon/data/cnn_1.pickle', 'rb') as fr:
  CNN_1_X = pickle.load(fr)
  CNN_1_Y = pickle.load(fr)
with open('/content/drive/Shareddrives/Dacon/data/lstm_1.pickle', 'rb') as fr:
  LSTM_1_X = pickle.load(fr)
  LSTM_1_Y = pickle.load(fr)  

with open('/content/drive/Shareddrives/Dacon/data/cnn_2.pickle', 'rb') as fr:
  CNN_2_X = pickle.load(fr)
  CNN_2_Y = pickle.load(fr)
with open('/content/drive/Shareddrives/Dacon/data/lstm_2.pickle', 'rb') as fr:
  LSTM_2_X = pickle.load(fr)
  LSTM_2_Y = pickle.load(fr)  

In [ ]:
CNN_1_X = np.array(CNN_1_X)
CNN_1_Y = np.array(CNN_1_Y)

In [ ]:
np.stack([CNN_1_X, LSTM_1_X])
np.stack([CNN_1_Y[0], LSTM_1_Y[0]]).shape

(2, 52128, 48)

In [ ]:
train_1_x = []
train_1_y = []
for i in range(9):
  temp_x = []
  temp_y = []
  for j in range(52125):
    temp_x.append(np.vstack([CNN_1_X[i][j], LSTM_1_X[i][j]]).transpose() )
    temp_y.append(np.vstack([CNN_1_Y[i][j]]).transpose() )
  train_1_x.append(temp_x)
  train_1_y.append(temp_y)

In [ ]:
train_1_x = np.array(train_1_x)
train_1_y = np.array(train_1_y).reshape(9, 52125, 48)
train_1_x.shape, train_1_y.shape

((9, 52125, 48, 2), (9, 52125, 48))

In [ ]:
train_1_x[0].shape

(52125, 48, 2)

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.backend import mean, maximum

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, restore_best_weights=True, patience=20)

def quantile_loss(q, y, pred):
  err = (y-pred)
  return mean(maximum(q*err, (q-1)*err), axis=-1)

def Stacking(train_x, train_y):
  ST_models = []

  for q in range(9) :
    X_train = train_x[q]
    y_train = train_y[q]
    print("★★★★★★★★★★  ", q, "  ★★★★★★★★★★")
#==============================================================================================================================================
    MLP_model = tf.keras.Sequential([
                                # tf.keras.layers.Dense(units=6, activation='tanh', input_shape=(1,)),
                                tf.keras.layers.Dense(units=48, input_shape=(48, 2)),
                                tf.keras.layers.Dense(units=24),
                                tf.keras.layers.Dense(units=12),
                                tf.keras.layers.Dense(units=6),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense(units=100),                            
                                tf.keras.layers.Dense(units=48, activation='relu'),
    ])

    MLP_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0007), loss=lambda y,pred: quantile_loss((q+1)*0.1,y,pred))
    #MLP_model.summary()
    
    MLP_model.fit(X_train, y_train, epochs=1000, batch_size=2000, validation_split=0.25, callbacks=[es])

    ST_models.append(MLP_model)

  return ST_models

In [ ]:
X_train = train_1_x[0]
y_train = train_1_y[0]
X_train.shape, y_train.shape

((52125, 48, 2), (52125, 48))

In [ ]:
import tensorflow as tf

Level1_1 = Stacking(train_1_x, train_1_y)

In [ ]:
train_2_x = []
train_2_y = []
for i in range(9):
  temp_x = []
  temp_y = []
  for j in range(52125):
    temp_x.append(np.vstack([CNN_2_X[i][j], LSTM_2_X[i][j]]).transpose() )
    temp_y.append(np.vstack([CNN_2_Y[i][j]]).transpose() )
  train_2_x.append(temp_x)
  train_2_y.append(temp_y)

In [ ]:
train_2_x = np.array(train_2_x)
train_2_y = np.array(train_2_y).reshape(9, 52125, 48)
train_2_x.shape, train_2_y.shape

((9, 52125, 48, 2), (9, 52125, 48))

In [ ]:
Level1_2 = Stacking(train_2_x, train_2_y)

# PREDICT

In [ ]:
test_cnn = pd.read_csv('ensemble_cnn.csv')

In [ ]:
Day1 = test_cnn[test_cnn['id'].apply(lambda x : True if 'Day7' in x else False)]
Day2 = test_cnn[test_cnn['id'].apply(lambda x : True if 'Day8' in x else False)]

In [ ]:
test_data_1 = []

test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.1']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.2']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.3']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.4']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.5']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.6']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.7']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.8']).reshape(81, 48) ))
test_data_1.append(scalers['TARGET'].transform(np.array(Day1['q_0.9']).reshape(81, 48) ))

test_data_2 = []

test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.1']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.2']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.3']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.4']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.5']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.6']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.7']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.8']).reshape(81, 48) ))
test_data_2.append(scalers['TARGET'].transform(np.array(Day1['q_0.9']).reshape(81, 48) ))

In [ ]:
test_data_1 = np.array(test_data_1)
test_data_2 = np.array(test_data_2)
test_data_1.shape, test_data_2.shape

((9, 81, 48), (9, 81, 48))

In [ ]:
test_lstm = pd.read_csv('ensemble_lstm.csv')

In [ ]:
Day1 = test_lstm[test_lstm['id'].apply(lambda x : True if 'Day7' in x else False)]
Day2 = test_lstm[test_lstm['id'].apply(lambda x : True if 'Day8' in x else False)]

In [ ]:
test_data_3 = []

test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.1']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.2']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.3']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.4']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.5']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.6']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.7']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.8']).reshape(81, 48) ))
test_data_3.append(scalers['TARGET'].transform(np.array(Day1['q_0.9']).reshape(81, 48) ))

test_data_4 = []

test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.1']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.2']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.3']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.4']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.5']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.6']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.7']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.8']).reshape(81, 48) ))
test_data_4.append(scalers['TARGET'].transform(np.array(Day1['q_0.9']).reshape(81, 48) ))

In [ ]:
test_data_3 = np.array(test_data_3)
test_data_4 = np.array(test_data_4)
test_data_3.shape, test_data_4.shape

((9, 81, 48), (9, 81, 48))

In [ ]:
test_X_day1 = []

for i in range(9):
  temp_x = []
  for j in range(81):
    temp_x.append(np.vstack([test_data_1[i][j], test_data_3[i][j]]).transpose() )
    
  test_X_day1.append(temp_x)

test_X_day2 = []

for i in range(9):
  temp_x = []
  for j in range(81):
    temp_x.append(np.vstack([test_data_2[i][j], test_data_4[i][j]]).transpose() )
    
  test_X_day2.append(temp_x)

In [ ]:
np.array(test_X_day1).shape

(9, 81, 48, 2)

In [ ]:
def mlp_predict(test_x, Level1_1):
  ST_pred = pd.DataFrame()
  quantiles = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

  for i in range(9):
    pred = Level1_1[i].predict(test_x[i])

    pred = pd.Series(pred.flatten())

    ST_pred = pd.concat([ST_pred, pred],axis=1)

  ST_pred.columns=quantiles

  return ST_pred

In [ ]:
MLP_result_1 = mlp_predict(np.array(test_X_day1), Level1_1)
MLP_result_1 = scalers['TARGET'].inverse_transform(MLP_result_1)
MLP_result_1 = pd.DataFrame(MLP_result_1)

In [ ]:
MLP_result_2 = mlp_predict(np.array(test_X_day2), Level1_2)
MLP_result_2 = scalers['TARGET'].inverse_transform(MLP_result_2)
MLP_result_2 = pd.DataFrame(MLP_result_2)

In [ ]:
submission = pd.read_csv('/content/drive/Shareddrives/Dacon/sample_submission.csv')
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = MLP_result_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = MLP_result_2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
submission.to_csv('Final.csv')